In [1]:
from pathlib import Path
import datetime as dt
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np

OUT_DIR = Path('../output')

In [2]:
BLANK = '?'
VOLUNTEERS_BY_WEEKDAY = {
    'Wednesday': ['Tommy', 'Etienne'],
    'Thursday': ['Sanna', 'Finlay'],
    'Sunday': ['Alex M', BLANK],
    }
MEETING_TIME_U = ('Sunday', 1, '15:00', '16:00')
MEETING_TIME_W = ('Wednesday', 1, '19:30', '20:30')
MEETING_TIME_R = ('Thursday', 1, '19:30', '20:30')

def build_roster(year, month, volunteers_by_weekday=VOLUNTEERS_BY_WEEKDAY, 
  meeting_time=None, blank=BLANK):
    """
    Given a year (integer) and a month (integer), build the roster for
    that year and month as a DataFrame with the columns
    
    - date: date of a shift
    - weekday: weekday of the shift, e.g. 'Sunday'
    - volunteers: names of volunteers working the shift
    - notes: any notes about the shift
    
    Also give a dictionary of volunteers by weekday (see example above)
    to seed the roster and a optionally a meeting time (see example above).
    """
    # Create roster dates
    d1 = dt.date(year, month, 1)
    d2 = d1 + relativedelta(months=1)
    ix = pd.date_range(d1, d2, closed='left')
    f = pd.DataFrame(index=ix)
    f['date'] = f.index.strftime('%Y-%m-%d')
    f['weekday'] = f.index.weekday_name
    
    # Assign volunteers
    f['volunteers'] = f['weekday'].map(lambda x:
      ' & '.join(volunteers_by_weekday.get(x, [BLANK, BLANK])))
    
    # Restrict to given weekdays
    cond = f['weekday'].isin(volunteers_by_weekday.keys())
    f = f[cond].copy()
    
    # Add notes, including staff meeting.
    f['notes'] = ''
    if meeting_time is not None:
        cond = f['weekday'] == meeting_time[0]
        cond &= f.index.day >= 7*(meeting_time[1] - 1)
        cond &= f.index.day <= 7*meeting_time[1]
        f.loc[cond, 'notes'] = 'Staff meeting {!s}--{!s}'.format(*meeting_time[2:])
    
    return f.reset_index(drop=True)

def get_year_month(roster, format="%Y%m"):
    """
    Given a roster, return the year and month it pertains to 
    in the given format.
    """
    return pd.to_datetime(roster['date']).dt.strftime(format).iat[0]

def to_markdown(df, path):
    """
    Save the given data frame as a Markdown table located
    at the given path.
    """
    path = Path(path)
    if not path.parent.exists():
        path.parent.mkdir(parents=True)
    
    # Get column names
    cols = df.columns

    # Create a new DataFrame with just the markdown
    # strings
    df2 = pd.DataFrame([['---',]*len(cols)], columns=cols)

    #Create a new concatenated DataFrame
    df3 = pd.concat([df2, df])

    #Save as markdown
    df3.to_csv(str(path), sep="|", index=False)


In [3]:
roster = build_roster(2017, 11, meeting_time=None)
print(roster)

date = get_year_month(roster)
path = OUT_DIR/'roster_{!s}.md'.format(date)
to_markdown(roster, path)

          date    weekday       volunteers notes
0   2017-11-01  Wednesday  Tommy & Etienne      
1   2017-11-02   Thursday   Sanna & Finlay      
2   2017-11-05     Sunday       Alex M & ?      
3   2017-11-08  Wednesday  Tommy & Etienne      
4   2017-11-09   Thursday   Sanna & Finlay      
5   2017-11-12     Sunday       Alex M & ?      
6   2017-11-15  Wednesday  Tommy & Etienne      
7   2017-11-16   Thursday   Sanna & Finlay      
8   2017-11-19     Sunday       Alex M & ?      
9   2017-11-22  Wednesday  Tommy & Etienne      
10  2017-11-23   Thursday   Sanna & Finlay      
11  2017-11-26     Sunday       Alex M & ?      
12  2017-11-29  Wednesday  Tommy & Etienne      
13  2017-11-30   Thursday   Sanna & Finlay      
